In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [2]:

import cv2
import math

import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async



In [3]:
import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 
from Mouvement import*
from vision import* 
from optimal_path import* 



map_lenght = 5
nb_of_square_by_side = 50
current = 1

In [4]:
global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)
kalman_bool = False
#pourcentage_reduc = 1


George = Robot()
Lausanne = Map(map_lenght, nb_of_square_by_side)

#VideoCap = vision_initialization()
VideoCap=cv2.VideoCapture(0)

print('Hello World')

####################################
## INIT
####################################

# Prend une première image
ret, frame=VideoCap.read()

# Set the pourcentage value between the nb of pixel and nb of square by side
Lausanne.set_pourcentage(frame)

# Set the robot goal, position and angle
goal = init_goal(frame, Lausanne.get_pourcentage())

pos_robot = George.get_pos()
while (pos_robot[0] < 0 & pos_robot[1] < 0):
    pos_robot, angle = update(frame, Lausanne.get_pourcentage())  # fonction qui retourne la position angle etc....

George.set_angle(angle)
George.set_goal(goal)
George.set_start_pos(pos_robot)
George.set_pos(pos_robot)
print ("goal", goal)
print ("start position", pos_robot)


####################################
## while
####################################


kalman_bool = False


print('while start')
while True:#(George.get_pos() != George.get_goal()):
    
    # Read the camera
    ret, frame=VideoCap.read()
    
    # Reupere la position et angle --> XAV
    #x_robot, y_robot,angle_robot = update(frame, Lausanne.get_pourcentage(), kalman_bool)
    pos_robot,angle_robot = update(frame, Lausanne.get_pourcentage())
    George.set_pos(pos_robot)
    George.set_angle(angle_robot)
    print ('position',pos_robot ,angle_robot)
    display(frame,1,1,1)
    
    
    # Quit when we press q
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        break
     
    sleep(0.1)

print("The End")
    

Hello World
 la position goal
[array([ 71, 143]), array([86, 52]), array([393, 222]), array([409, 107])]
goal (7.395833333333334, 35.10416666666667)
start position (0.0, 0.0)
while start
position (0.0, 0.0) 0
position (0.0, 0.0) 0
position (0.0, 0.0) 0
position (0.0, 0.0) 0
robot detected
if
position (19.166666666666668, 17.8125) 1.4767350921669102
robot detected
if
position (19.479166666666668, 16.979166666666668) 1.4352686128093959
robot detected
if
position (19.166666666666668, 16.770833333333336) 1.473543128543331
robot detected
if
position (19.479166666666668, 17.083333333333336) 1.4382447944982226
robot detected
if
position (19.375, 16.770833333333336) 1.3986055122719576
robot detected
if
position (19.479166666666668, 17.1875) 1.3986055122719576
robot detected
if
position (19.375, 17.604166666666668) 1.4711276743037347
robot detected
if
position (20.0, 16.5625) 1.295886485314224
robot detected
if
position (19.479166666666668, 17.5) 1.4288992721907328
robot detected
if
position (1

robot detected
if
position (19.166666666666668, 16.875) 1.4758446204521403
robot detected
if
position (19.479166666666668, 16.770833333333336) 1.4758446204521403
robot detected
if
position (18.854166666666668, 12.395833333333334) -1.7126933813990606
robot detected
if
position (19.479166666666668, 16.875) 1.4321565896607384
robot detected
if
position (19.270833333333336, 16.5625) 1.5208379310729538
robot detected
if
position (19.270833333333336, 16.666666666666668) 1.4432867685796584
robot detected
if
position (18.854166666666668, 12.291666666666668) -1.7126933813990606
robot detected
if
position (19.375, 16.875) 1.4288992721907328
robot detected
if
position (19.479166666666668, 16.666666666666668) 1.4056476493802699
robot detected
if
position (18.854166666666668, 12.395833333333334) -1.6892862859530344
robot detected
if
position (19.583333333333336, 17.083333333333336) 1.4164775508687777
robot detected
if
position (19.166666666666668, 16.875) 1.4994888620096063
robot detected
if
positi

robot detected
if
position (19.479166666666668, 16.770833333333336) 1.5475447039844337
robot detected
if
position (19.479166666666668, 16.770833333333336) 1.5480729659532555
robot detected
if
position (19.479166666666668, 16.770833333333336) 1.5480729659532555
robot detected
if
position (19.479166666666668, 16.666666666666668) 1.5475447039844337
robot detected
if
position (19.479166666666668, 16.666666666666668) 1.5232132235179132
robot detected
if
position (19.479166666666668, 16.770833333333336) 1.5243181954438936
robot detected
if
position (19.479166666666668, 16.666666666666668) 1.4994888620096063
robot detected
if
position (19.479166666666668, 16.666666666666668) 1.5475447039844337
robot detected
if
position (19.583333333333336, 16.770833333333336) 1.5243181954438936
robot detected
if
position (19.583333333333336, 16.770833333333336) 1.5253730473733196
robot detected
if
position (19.583333333333336, 16.770833333333336) 1.5011417530663285
robot detected
if
position (19.6875, 16.770

robot detected
if
position (42.60416666666667, 19.791666666666668) -0.8760580505981933
robot detected
if
position (41.04166666666667, 21.666666666666668) 2.356194490192345
robot detected
if
position (41.145833333333336, 21.979166666666668) 2.312743594800814
position (41.145833333333336, 21.979166666666668) 2.312743594800814
robot detected
if
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7022569315090071
position (42.5, 20.729166666666668) -0.7

position (45.72916666666667, 20.0) 0.4746968830801752
position (45.72916666666667, 20.0) 0.4746968830801752
position (45.72916666666667, 20.0) 0.4746968830801752
position (45.72916666666667, 20.0) 0.4746968830801752
robot detected
if
position (47.5, 17.395833333333336) 0.11202896242598764
robot detected
if
position (47.708333333333336, 17.604166666666668) 0.13335025417155275
robot detected
if
position (48.020833333333336, 17.395833333333336) 0.10673567264913629
robot detected
if
position (48.22916666666667, 17.395833333333336) 0.1144403052700638
robot detected
if
position (48.333333333333336, 17.604166666666668) 0.12435499454676072
robot detected
if
position (48.958333333333336, 17.5) 0.10487693873023407
robot detected
if
position (48.85416666666667, 17.604166666666668) 0.11649146069465083
robot detected
if
position (48.75, 17.8125) 0.13888501552021276
robot detected
if
position (48.958333333333336, 17.916666666666668) 0.14631528800503077
robot detected
if
position (48.958333333333336,

...

position (30.208333333333336, 26.25) -0.05258306161094078
position (30.208333333333336, 26.25) -0.05258306161094078
position (30.208333333333336, 26.25) -0.05258306161094078
position (30.208333333333336, 26.25) -0.05258306161094078
position (30.208333333333336, 26.25) -0.05258306161094078
position (30.208333333333336, 26.25) -0.05258306161094078
position (30.208333333333336, 26.25) -0.05258306161094078
robot detected
if
position (56.770833333333336, 29.375) 1.306832603169192
robot detected
if
position (56.770833333333336, 29.375) 1.306832603169192
robot detected
if
position (56.770833333333336, 29.375) 1.3321870042866957
robot detected
if
position (56.770833333333336, 29.375) 1.306832603169192
robot detected
if
position (56.770833333333336, 29.375) 1.3321870042866957
robot detected
if
position (56.770833333333336, 29.375) 1.306832603169192
robot detected
if
position (56.770833333333336, 29.375) 1.306832603169192
robot detected
if
position (56.770833333333336, 29.375) 1.3321870042866957

robot detected
if
position (60.41666666666667, 49.583333333333336) 1.0666303653157436
robot detected
if
position (60.520833333333336, 49.6875) 1.0447299077981331
robot detected
if
position (60.625, 49.79166666666667) 1.034985088834433
robot detected
if
position (60.520833333333336, 49.895833333333336) 1.0951011079399045
robot detected
if
position (60.520833333333336, 49.6875) 1.0303768265243125
robot detected
if
position (60.625, 49.79166666666667) 1.0209431441183117
robot detected
if
position (60.41666666666667, 49.79166666666667) 1.082462375738491
robot detected
if
position (60.41666666666667, 49.79166666666667) 1.082462375738491
robot detected
if
position (60.625, 49.79166666666667) 1.034985088834433
robot detected
if
position (60.3125, 49.583333333333336) 1.1071487177940904
robot detected
if
position (60.41666666666667, 49.6875) 1.0691922726057763
robot detected
if
position (60.520833333333336, 49.79166666666667) 1.0714496051147666
robot detected
if
position (60.520833333333336, 49

robot detected
if
position (65.0, 41.770833333333336) 1.4876550949064553
robot detected
if
position (65.0, 41.875) 1.4659193880646626
robot detected
if
position (65.0, 41.770833333333336) 1.4876550949064553
robot detected
if
position (64.89583333333334, 41.770833333333336) 1.5167948264189284
robot detected
if
position (64.89583333333334, 41.875) 1.5437758776076318
robot detected
if
position (65.0, 41.875) 1.492012365805753
robot detected
if
position (64.89583333333334, 41.770833333333336) 1.5167948264189284
robot detected
if
position (65.0, 41.875) 1.5182132651839548
robot detected
if
position (65.0, 41.875) 1.4659193880646626
robot detected
if
position (64.89583333333334, 41.875) 1.5182132651839548
robot detected
if
position (65.0, 41.770833333333336) 1.5152978215491797
robot detected
if
position (65.0, 41.875) 1.5167948264189284
robot detected
if
position (65.0, 41.770833333333336) 1.5167948264189284
robot detected
if
position (65.0, 41.770833333333336) 1.5167948264189284
robot detec

position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668) 1.299849476456476
position (16.354166666666668, 22.291666666666668

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 1: invalid continuation byte